In [1]:
'''

用戶菜單功能介紹

    用戶能透過點擊菜單，進行我方希冀之業務功能。
    
流程
    準備菜單的圖面設定檔
    讀取安全設定檔上的參數
    將菜單設定檔傳給Line
    對Line上傳菜單照片
    檢視現有的菜單
    將菜單與用戶做綁定
    將菜單與用戶解除綁定
    刪除菜單

'''

'\n\n用戶菜單功能介紹\n\n    用戶能透過點擊菜單，進行我方希冀之業務功能。\n    \n流程\n    準備菜單的圖面設定檔\n    讀取安全設定檔上的參數\n    將菜單設定檔傳給Line\n    對Line上傳菜單照片\n    檢視現有的菜單\n    將菜單與用戶做綁定\n    將菜單與用戶解除綁定\n    刪除菜單\n\n'

In [1]:
'''
菜單設定檔

    設定圖面大小、按鍵名與功能
    
'''

menuRawData="""
{
  "size": {
    "width": 2500,
    "height": 1686
  },
  "selected": true,
  "name": "foodMenu",
  "chatBarText": "查看更多資訊",
  "areas": [
    {
      "bounds": {
        "x": 32,
        "y": 369,
        "width": 1198,
        "height": 461
      },
      "action": {
        "type": "message",
        "text": "[::text:]本次交易"
      }
    },
    {
      "bounds": {
        "x": 1248,
        "y": 369,
        "width": 1167,
        "height": 449
      },
      "action": {
        "type": "message",
        "text": "[::text:]歷史交易"
      }
    },
    {
      "bounds": {
        "x": 49,
        "y": 866,
        "width": 1192,
        "height": 462
      },
      "action": {
        "type": "message",
        "text": "[::flex:]促銷食材"
      }
    },
    {
      "bounds": {
        "x": 1247,
        "y": 866,
        "width": 1185,
        "height": 456
      },
      "action": {
        "type": "uri",
        "uri": "https://image.slidesharecdn.com/chinesehistory-cv-caocao-140730210730-phpapp01/95/-1-638.jpg?cb=1458796316"
      }
    }
  ]
}
"""

In [2]:
'''

讀取安全檔案內的字串，以供後續程式碼調用

'''
import json
secretFileContentJson=json.load(open("../line_secret_key",'r'))

print(secretFileContentJson.get("channel_access_token"))
print(secretFileContentJson.get("secret_key"))
print(secretFileContentJson.get("self_user_id"))


p8HLmCcB+KSzk93/zxumyxGec/9Ue022dj7noPluNs3jFVSdDLWP6N8njU+VpF9YL8IRtN/3YaxjcXkZJMBSbqt82yGSJjD885hForRc7zARCacpByMLI2DDEHS0zO2AiKYh78NhhoDlVRVcIh5dmwdB04t89/1O/w1cDnyilFU=
2d6a6914319f6af6dcd6a408e9f2c95d
U7173dc299af6ac3fe7697a75c2a280ad


In [3]:
'''

將先前準備的菜單設定檔，以Post消息寄發給Line

    設定Line的遠端位置
    設定消息的基本安全憑證
    寄發消息，並取得回應之Id
    

'''

import requests

# 載入設定檔
menuJson=json.loads(menuRawData)

# 上傳到 Line
createMenuEndpoint = 'https://api.line.me/v2/bot/richmenu'

# 將認證碼 channel_access_token 放到封包的表頭上
createMenuRequestHeader={'Content-Type':'application/json','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

#print(createMenuRequestHeader)

# 傳給 Line
lineCreateMenuResponse = requests.post(createMenuEndpoint,headers=createMenuRequestHeader,data=json.dumps(menuJson))

# 印出結果
print(lineCreateMenuResponse)
print(lineCreateMenuResponse.text)


<Response [200]>
{"richMenuId":"richmenu-61b4d43cd84d82e150e1d2277d96253e"}


In [4]:
'''

將先前準備的菜單照片，以Post消息寄發給Line
    取得上面設定檔的Id
    設定Line的遠端位置
    設定消息的基本安全憑證
    上傳照片，並取得回傳成果

'''

# 取得功能選單的 id
uploadRichMenuId=json.loads(lineCreateMenuResponse.text).get("richMenuId")
print(uploadRichMenuId)
#'https://api.line.me/v2/bot/richmenu/{richMenuId}/content'

# 設定上傳位置
uploadMenuEndpoint='https://api.line.me/v2/bot/richmenu/%s/content' % uploadRichMenuId
print(uploadMenuEndpoint)

# 為封包設定認證的公司章 (channel_access_token)
uploadMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 讀取圖片
uploadImageFile=open("../food/menu2.jpg",'rb')

# 上傳圖片
lineUploadMenuResponse=requests.post(uploadMenuEndpoint,headers=uploadMenuRequestHeader,data=uploadImageFile)

# 觀看結果
print(lineUploadMenuResponse)
print(lineUploadMenuResponse.text)

richmenu-61b4d43cd84d82e150e1d2277d96253e
https://api.line.me/v2/bot/richmenu/richmenu-61b4d43cd84d82e150e1d2277d96253e/content
<Response [200]>
{}


In [5]:
'''

將選單綁定到特定用戶身上
    取出上面得到的菜單Id及用戶id
    設定Line的遠端位置
    設定消息的基本安全憑證
    發送消息告知

'''

# https://api.line.me/v2/bot/user/{userId}/richmenu/{richMenuId}

# 取得菜單的 id
linkRichMenuId=json.loads(lineCreateMenuResponse.text).get("richMenuId")

#將 user_id 和 richmenu_id 組合成一個網址
linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (secretFileContentJson["self_user_id"], linkRichMenuId)
print(linkMenuEndpoint)

# 為封包設定安全標籤(蓋公司章)
linkMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 將需求傳給 Line
lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
print(lineLinkMenuResponse)
print(lineLinkMenuResponse.text)

https://api.line.me/v2/bot/user/U7173dc299af6ac3fe7697a75c2a280ad/richmenu/richmenu-61b4d43cd84d82e150e1d2277d96253e
<Response [200]>
{}


In [17]:
'''

檢視用戶目前所綁定的菜單
    取出用戶id
    設定Line的遠端位置
    設定消息的基本安全憑證
    發送消息告知

'''

#  https://api.line.me/v2/bot/user/{userId}/richmenu

# 取出用戶id，設定Line的遠端位置
userMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu' % (secretFileContentJson["self_user_id"])
print(userMenuEndpoint)

# 設定消息的基本安全憑證
userMenuRequestHeader={'Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 發送消息告知
lineUserMenuResponse=requests.get(userMenuEndpoint,headers=userMenuRequestHeader)
print(lineUserMenuResponse)
print(lineUserMenuResponse.text)


https://api.line.me/v2/bot/user/U7173dc299af6ac3fe7697a75c2a280ad/richmenu
<Response [404]>
{"message":"the user has no richmenu","details":[]}


In [14]:
'''

解除選單與特定用戶的綁定
    取出用戶id
    設定Line的遠端位置
    設定消息的基本安全憑證
    發送消息告知

'''

# 取出用戶id，設定Line的遠端位置
userMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu' % (secretFileContentJson["self_user_id"])
print(userMenuEndpoint)

# 設定消息的基本安全憑證
userMenuRequestHeader={'Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}

# 發送消息告知
lineUnregisterUserMenuResponse=requests.delete(userMenuEndpoint,headers=userMenuRequestHeader)
print(lineUnregisterUserMenuResponse)
print(lineUnregisterUserMenuResponse.text)

https://api.line.me/v2/bot/user/U7173dc299af6ac3fe7697a75c2a280ad/richmenu
<Response [200]>
{}


In [15]:
'''

檢視帳號內，有哪些選單
    設定Line的遠端位置
    設定消息的基本安全憑證
    發送消息告知

'''

# 設定Line的遠端位置
listMenuEndpoint="https://api.line.me/v2/bot/richmenu/list"

# 設定消息的基本安全憑證
listMenuHeader={'Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}    

# 發送消息告知
lineLisrMenuResponse = requests.get(listMenuEndpoint,headers=listMenuHeader)
print(lineLisrMenuResponse.text)

{"richmenus":[{"richMenuId":"richmenu-4e6a0410ec47f5933bfdf78ca480fd8e","name":"區塊鏈自定義菜單","size":{"width":2500,"height":1686},"chatBarText":"查看更多資訊","selected":true,"areas":[{"bounds":{"x":5,"y":0,"width":824,"height":850},"action":{"type":"message","text":"[::text:]傳統交易"}},{"bounds":{"x":0,"y":850,"width":825,"height":818},"action":{"type":"message","text":"[::text:]第三方公證人"}},{"bounds":{"x":829,"y":5,"width":871,"height":849},"action":{"type":"message","text":"[::text:]多位公證人"}},{"bounds":{"x":825,"y":854,"width":875,"height":814},"action":{"type":"message","text":"[::text:]多組織多位公證人"}},{"bounds":{"x":1700,"y":0,"width":800,"height":858},"action":{"type":"message","text":"[::text:]教學訊息"}},{"bounds":{"x":1700,"y":858,"width":800,"height":810},"action":{"type":"message","text":"[::text:]more"}}]}]}
